<a href="https://colab.research.google.com/github/afnf33/emoTale/blob/master/kobert_ending_finale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브와 연동합니다
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# 1. 데이터 불러오기, 데이터 전처리

In [ ]:
import pandas as pd

sad = pd.read_excel('/content/drive/My Drive/data/tweet_list_슬픔 1~5000.xlsx')
happy = pd.read_excel('/content/drive/My Drive/data/tweet_list_기쁨 labeling_완료.xlsx')
annoy = pd.read_excel('/content/drive/My Drive/data/tweet_list_짜증_완료.xlsx')
fear = pd.read_excel('/content/drive/My Drive/data/tweet_list_공포_완료.xlsx')

sad2 = pd.read_csv('/content/drive/My Drive/data/추가_슬픔.csv')
happy2 = pd.read_csv('/content/drive/My Drive/data/추가_기쁨.csv')
annoy2 = pd.read_csv('/content/drive/My Drive/data/추가_분노.csv')
fear2 = pd.read_csv('/content/drive/My Drive/data/추가_공포1.txt', encoding='utf8')



In [ ]:
for i in [sad, happy, annoy, fear, sad2, happy2, annoy2, fear2]:
  print(i.head())

   Unnamed: 0                                           raw_text
0        5610  가난한 아버지가 가련한 아들을 껴안고 잠든 밤 마른 이불과 따끈따끈한 요리를 꿈꾸며...
1        2189                               가담도 완결나면 유료화되겠지 좀 슬픔
2        1843               가려진시간 뭔데 존나 슬픔 하아 ㅠㅠ 아 시발 진짜 시발 ㅠㅠㅠㅠ
3         497  가리고 싶은 감정이 참 많았어요 뭐라 부르기도 싫고 그냥 그런 것이 없는 사람이 되...
4        2165      가만보면 블루님은 다 귀엽다고 하시는데 본인이 귀여운지는 모르시는 거 같아요 슬픔
   Unnamed: 0                                           raw_text
0         342      감정적인 거 감정에 대고 호소하거나 분노나 기쁨 등을 확실히 표현하는 캐 사랑함다
1         648         그 순간이면 마치 우리는 마법에 걸린 것처럼 기쁨과 환희의 맛을 느끼곤 했지
2        2284   그냥 답님이 나가단에게 해주셨던 만큼 돌려받으셨으면 좋겠어 웃음 주신만큼행복 주신...
3        2394            그러게나 말입니다 이렇게 소리소문도없이 은근하게 이렇게 기쁨을 주네요 
4         178                                         그렇다고 하니 기쁨
   Unnamed: 0                                           raw_text
0           0  여름만큼 머리카락이 거슬리고 짜증나는 계절을 없을듯숏으로 잘라버리고 싶네 중장르 얘...
1           2                                        아짜증나 진짜 자야지
2           3  국수 먹을때 면치기

In [ ]:
sad['emotion'] = '슬픔'
happy['emotion'] = '기쁨'
annoy['emotion'] = '분노'
fear['emotion'] = '공포'

happy2.columns = ['Unnamed: 0', 'raw_text', 'emotion']

In [ ]:
sentence = pd.concat([sad, happy, annoy, fear], axis=0, ignore_index=True)
for i in ['슬픔','기쁨','분노','공포']:
  print(i,len(sentence[sentence['emotion'] == i]))
sentence.head()

슬픔 2696
기쁨 2247
분노 8250
공포 1098


,Unnamed: 0,raw_text,emotion
0,5610,가난한 아버지가 가련한 아들을 껴안고 잠든 밤 마른 이불과 따끈따끈한 요리를 꿈꾸며...,슬픔
1,2189,가담도 완결나면 유료화되겠지 좀 슬픔,슬픔
2,1843,가려진시간 뭔데 존나 슬픔 하아 ㅠㅠ 아 시발 진짜 시발 ㅠㅠㅠㅠ,슬픔
3,497,가리고 싶은 감정이 참 많았어요 뭐라 부르기도 싫고 그냥 그런 것이 없는 사람이 되...,슬픔
4,2165,가만보면 블루님은 다 귀엽다고 하시는데 본인이 귀여운지는 모르시는 거 같아요 슬픔,슬픔


## 데이터 개수 확인 후 학습 개수를 동일하게 맞춰주기

In [ ]:
sad_3 = sad[:1000]
happy_3 = happy[:1000]
annoy_3 = annoy[:2400]
# 분노 감정의 경우, 2차 레이블링 데이터가 1000개 부족하여 1차 레이블링 데이터를 2000개 사용
sentence = pd.concat([sad_3, happy_3, annoy_3, fear], axis=0, ignore_index=True)
for i in ['슬픔','기쁨','분노','공포']:
  print(i,len(sentence[sentence['emotion'] == i]))
sentence.head()

슬픔 1000
기쁨 1000
분노 2400
공포 1098


,Unnamed: 0,raw_text,emotion
0,5610,가난한 아버지가 가련한 아들을 껴안고 잠든 밤 마른 이불과 따끈따끈한 요리를 꿈꾸며...,슬픔
1,2189,가담도 완결나면 유료화되겠지 좀 슬픔,슬픔
2,1843,가려진시간 뭔데 존나 슬픔 하아 ㅠㅠ 아 시발 진짜 시발 ㅠㅠㅠㅠ,슬픔
3,497,가리고 싶은 감정이 참 많았어요 뭐라 부르기도 싫고 그냥 그런 것이 없는 사람이 되...,슬픔
4,2165,가만보면 블루님은 다 귀엽다고 하시는데 본인이 귀여운지는 모르시는 거 같아요 슬픔,슬픔


In [ ]:
sentence = pd.concat([sentence, sad2, annoy2, happy2, fear2], axis=0, ignore_index=True)
data_for_train = sentence[sentence['emotion'] != '중립']
data_for_train = data_for_train[['raw_text','emotion']]
for i in ['슬픔','기쁨','분노','공포']:
  print(i,len(sentence[sentence['emotion'] == i]))

슬픔 4222
기쁨 4620
분노 4457
공포 4430


# 2. 모델 투입

In [ ]:
##################################################
#모델에 입력하기 위해 형식을 맞춰줍니다

def label(x):
  if x=='슬픔': return 0.0
  elif x=='기쁨': return 1.0
  elif x=='분노': return 2.0
  elif x=='공포': return 3.0
  else: return x

data_for_train["emotion"] = data_for_train["emotion"].apply(label)


dtls = [list(data_for_train.iloc[i,:]) for i in range(len(data_for_train))]
dtls[:10] #형식이 통일되었습니다




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


[[' 가이 자첫이 막공이다슬픔', 0.0],
 [' 가짜슬픔 진짜슬픔 ', 0.0],
 [' 같이 여행자를 올려다보다가 이내 시선을 떨구며 당신의 손을 부드럽게 잡고는 아무 말도 하지 않았다 복잡한 감정이 얽힌 슬픔을 말로 표현하기는 너무 어려웠으니까 ',
  0.0],
 [' 개똥이가 슬픔을 못 이겨 똥찬이 옷에 붙었습니다 ', 0.0],
 [' 걔 보내고주밀레가 상흔 입은 건 확실한데 정확한 감정은 뭐였을까 연민 동정 슬픔', 0.0],
 [' 고앵 오지게 좋아하는데 고앵 알러지 잇음ㅠ 재채기 하고 콧물 줄줄 흐르고 난리남 슬픔', 0.0],
 [' 고함치던 현세의 경계에서 어리석은 당신은 울고 있었어향기가 여름바람에 이끌어져서 슬픔 따위는 없어', 0.0],
 [' 공근께서 하신 말은 없더이까 슬픔이 언뜻 비춰오는 모습에 쉽사리 말을 꺼내지 못한다', 0.0],
 [' 괴담 가족이 한 기이한 행동의 진짜 의미일찍 깨닫지 못한 슬픔 깨달은 후의 괴로움 ', 0.0],
 [' 교수 테이블에 항상 싱싱한 내음을 보이던 꽃다발들은 보이지 않게 된 지 오래입니다 누군가와의 이별의 슬픔은 할 만큼 했다는 듯이 치워졌습니다 ',
  0.0]]

In [ ]:
# 필요한 모듈을 설치합니다
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch #원래 ==1.3.1

     |████████████████████████████████| 711.7MB 26kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 3.2MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588540 sha256=adf2b237f2a1c7c9003f13d47be0e3c8b3e765d771ae5371ed061bb23f867698
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 3.5MB/s 
     |████████████████████████████████| 317kB 2.9MB/s 
     |████████████████████████████████| 890kB 30.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=323f3ebf440db61b9ba55ab3a9662cf5fd1bf2f20b1538278ebd63d946c1a605
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacrem

In [ ]:
#SKT에서 공개한 KoBERT 모델을 불러옵니다 
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-fr6ub3tx
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-fr6ub3tx
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12820 sha256=aa38926ba0f78a05bc62595f3f05e799f7161dab76b7d6a266e0ee063ac0b2c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-rcxs91_k/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

from kobert.utils import get_tokenizer 
from kobert.pytorch_kobert import get_pytorch_kobert_model 

from transformers import AdamW 
from transformers.optimization import WarmupLinearSchedule 

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
#koBERT의 토크나이저를 사용합니다
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# 3. 학습

In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1 
log_interval = 200 
learning_rate =  5e-5

In [ ]:
# train, validation, test set을 나눠주세요
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(dtls, test_size=0.2, random_state=123)

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
#모델을 만들고 GPU 사용 설정을 해줍니다
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
#옵티마이저와 손실함수 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

#정확도를 계산하기 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
#학습 과정
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad() 
        token_ids = token_ids.long().to(device) 
        segment_ids = segment_ids.long().to(device) 
        valid_length= valid_length 
        label = label.long().to(device) 
        out = model(token_ids, valid_length, segment_ids) 
        loss = loss_fn(out, label) 
        loss.backward() 
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) 
        optimizer.step() 
        scheduler.step()  # Update learning rate schedule 
        train_acc += calc_accuracy(out, label)  
        if batch_id % log_interval == 0: 
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1))) 
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1))) 
    model.eval() #모델 평가 부분 
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


epoch 1 batch id 1 loss 1.421995759010315 train acc 0.296875
epoch 1 batch id 201 loss 0.0907863900065422 train acc 0.6977611940298507

epoch 1 train acc 0.8268580243854083



epoch 1 test acc 0.9655283505154639


epoch 2 batch id 1 loss 0.06711960583925247 train acc 0.96875
epoch 2 batch id 201 loss 0.0425533801317215 train acc 0.9684390547263682

epoch 2 train acc 0.9737032860824743



epoch 2 test acc 0.971139878163074


epoch 3 batch id 1 loss 0.1160702034831047 train acc 0.96875
epoch 3 batch id 201 loss 0.05675015598535538 train acc 0.9825870646766169

epoch 3 train acc 0.9856233891752577



epoch 3 test acc 0.9747100515463918


In [ ]:
# 차후 사용을 위해 학습된 모델을 저장했습니다
torch.save(model.state_dict(), 'drive/My Drive//kobert_ending_finale.pt')


# 3.5 모델 불러오기

In [ ]:
''' 
# 필요한 모듈을 설치합니다
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch #원래 ==1.3.1

#SKT에서 공개한 KoBERT 모델을 불러옵니다 
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

from kobert.utils import get_tokenizer 
from kobert.pytorch_kobert import get_pytorch_kobert_model 

from transformers import AdamW 
from transformers.optimization import WarmupLinearSchedule 

##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

#koBERT의 토크나이저를 사용합니다
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1 
log_interval = 200 
learning_rate =  5e-5



# 차후 사용시 저장해둔 모델을 불러옵니다
# 필요한 모듈을 다시 설치하고 불러와야 합니다. 또한 class와 def도 재선언 해야 합니다
'''
# #모델을 불러옵니다
# model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# model.load_state_dict(torch.load('drive/My Drive/kobert_ending_finale.pt'))
# model.eval()




BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

# 4.1 1번 평가

In [ ]:
import pandas as pd

angr = pd.read_excel('/content/drive/My Drive/data/tweet_list_화남.xlsx')
jonna = pd.read_excel('/content/drive/My Drive/data/tweet_list_존나 좋아.xlsx')
kyum = pd.read_excel('/content/drive/My Drive/data/tweet_list_극혐.xlsx')

for i in [angr, jonna, kyum]:
  print(i.head())

   Unnamed: 0                                           raw_text  Unnamed: 2
0         0.0                                생각해도 좀 화남 로우시류 뮈친놈ㅋ         1.0
1         NaN                                                NaN         NaN
2         2.0  진짜 생각하면 할수록 빡치고 마음아픔 왜 우리 태성이만 불행해 우리 태성이 오해 받...         1.0
3         3.0                                                NaN         NaN
4         4.0                                           그래서 더 화남         1.0
   Unnamed: 0                                           raw_text  Unnamed: 2
0           0  근데 조슈프라 좀ㄱㅊ은거같아 일단 관계가 어케 개선되도 조슈아가 프라우를 존나 싫어...           1
1           1  미구하라 여행용키트 나왔는데 정품으로 꽉꽉 채워서 받을 수 있대요 월말까지만 요 가...           0
2           2  이 아줌마 나한테 계속 살인마 키워놓고 큰소리친다고 니들땜에 우리 ㅇㅇ이 죽었다 소...           0
3           3  최윤모르는 사람이 봐도 아 얘는 겨울에 태어났을 거 같은디 라고 할 것 같은데 한여...           0
4           4  아 오팔은 좋은데 초코오팔은 취향아닌듯 존나삼 난역시 커팅보단 러프원석이 좋아 커팅...           1
   Unnamed: 0                                           raw_text  Unnamed: 2

In [ ]:
angr = angr[['raw_text']].dropna()
jonna = jonna[jonna['Unnamed: 2']==1]
jonna = jonna[['raw_text']].dropna()
kyum = kyum[['raw_text']].dropna()

angr.columns = ['Sentence']
jonna.columns = ['Sentence']
kyum.columns = ['Sentence']

for i in [angr, jonna, kyum]:
  print(i.head())


                                            Sentence
0                                생각해도 좀 화남 로우시류 뮈친놈ㅋ
2  진짜 생각하면 할수록 빡치고 마음아픔 왜 우리 태성이만 불행해 우리 태성이 오해 받...
4                                           그래서 더 화남
5  아직두 화남 로투킹을 왜 안 봣을가 트친놈들 다 볼 때 나도 봤어야 햇는데 뒷북으로...
6  머위님이 항상 군인들 처우 언급하는거 보면 한국 군머의 부실한 대우와 윗사람들과 트...
                                            Sentence
0  근데 조슈프라 좀ㄱㅊ은거같아 일단 관계가 어케 개선되도 조슈아가 프라우를 존나 싫어...
4  아 오팔은 좋은데 초코오팔은 취향아닌듯 존나삼 난역시 커팅보단 러프원석이 좋아 커팅...
5                                             뭐 존나좋아
6                       허흐허ㅓ으러러 레몬 존나 죠아 시바아아라르러루루ㅜㅜ
7  아 근데 진짜 빻았지만 왤케 짐승같은 거 좋냐 ㅋㅌㅋㅋㅋㅋㅋㅋ 보지 냄새맡고 킁킁대...
                                            Sentence
1                                  워싱 칭따오 씝쓉씝씝쓉씝씝 극혐
2                          좆극혐 시팔시팔 벌레는 근데 원래 다 극혐이야
3  길수현사랑이 이라면 제가 동찬 씨를 온전히 만큼 사랑하기 때문에 그가 날 사랑할 수...
4                                 직접말못하고 뒤에서 까는거 극혐 
5  아리발 갑자기 고딩 때ㅋㅋ 수학학원 애들이랑 같이 노래방 가서 바나나알러지원숭이 부...


In [ ]:
eval_data = pd.concat([angr, jonna, kyum], axis=0, ignore_index=True)
eval_data['emotion'] = pd.concat(
    [pd.Series(['annoy' for i in range(len(angr))]),
    pd.Series(['happy' for i in range(len(jonna))]),
    pd.Series(['fear' for i in range(len(kyum))])], axis=0, ignore_index=True
)
eval_data.head()

,Sentence,emotion
0,생각해도 좀 화남 로우시류 뮈친놈ㅋ,annoy
1,진짜 생각하면 할수록 빡치고 마음아픔 왜 우리 태성이만 불행해 우리 태성이 오해 받...,annoy
2,그래서 더 화남,annoy
3,아직두 화남 로투킹을 왜 안 봣을가 트친놈들 다 볼 때 나도 봤어야 햇는데 뒷북으로...,annoy
4,머위님이 항상 군인들 처우 언급하는거 보면 한국 군머의 부실한 대우와 윗사람들과 트...,annoy


In [ ]:
for i in range(len(eval_data['Sentence'])):
  if type(eval_data['Sentence'][i]) != str:
    eval_data['Sentence'][i] = str(eval_data['Sentence'][i])

In [ ]:
##################################################
# 평가용 Test_set을 모델에 입력하기 위해 형식을 맞춰줍니다

def label(x):
  if x=='sad': return 0.0
  elif x=='happy': return 1.0
  elif x=='annoy': return 2.0
  elif x=='fear': return 3.0
  else: return x

eval_data["emotion"] = eval_data["emotion"].apply(label)

dtls = [list(eval_data.iloc[i,:]) for i in range(len(eval_data))]
dtls[:10] #형식이 통일되었습니다



[[' 생각해도 좀 화남 로우시류 뮈친놈ㅋ', 2.0],
 ['진짜 생각하면 할수록 빡치고 마음아픔 왜 우리 태성이만 불행해 우리 태성이 오해 받고도 누나 생각해서 앞에서 예쁘게 웃고 아픈거 말도 안 하고 혼자 알아서 외국으로 비켜줫는데 결국 죽을때까지 좋아하는 누나 얼굴도 못 보고진짜 너무 화남 우리 태성이 하늘에선 절대 행복해야해',
  2.0],
 ['그래서 더 화남', 2.0],
 ['아직두 화남 로투킹을 왜 안 봣을가 트친놈들 다 볼 때 나도 봤어야 햇는데 뒷북으로 정주행 했다가 짱오오 알아벌인 색큩', 2.0],
 ['머위님이 항상 군인들 처우 언급하는거 보면 한국 군머의 부실한 대우와 윗사람들과 트러블로 전역한 것 같은데 진짜 우리나라를 위해 온 인재를 멍청하게 놓친 것 같아서 화남 분명 처우가 좋았더라면 머위님 아직도 전역 안하고 계속 군머에 있었을 사람임 생각하니까 또 빡치네 퍼킹 코리아',
  2.0],
 ['이렇게 매번 말씀을 하시는데도 무시를 까는건지 와닿는게 없어서인건지 지들 좆대로 구는거 화남 대가리 다 깨버려야 돼 ', 2.0],
 ['아니 나 그냥 화남 ㅣ발', 2.0],
 ['갱상도 졸라도이딴말 해가면서아직도 지역감정 따위나 찾는 개 샥히 들은 다들 모아다가 대구리를 싹다 짤라 버려야 한다 지저분한 걸래만도 못한 식히들 화남화남',
  2.0],
 ['씨브아알ㅋㅋ볼때마다좋은데화남 ', 2.0],
 ['아진짜 중딩때이후로 처음이다 안그래도 지금 장마때문에 일전 빨래도 안마르고있어서 이불 또 말릴ㅈ생각에ㅈ화나는데 오랜만에 예쁜 이불깔고자서 완전신나서 이이불 천년만년 깔고자야지 했었단말여ㅜㅜ화남',
  2.0]]

In [ ]:
data_test = BERTDataset(dtls, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
#정확도를 계산하기 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
# 해당 데이터에 대해 분류를 시작합니다
model.eval()
answer=[]
train_acc = 0.0
test_acc = 0.0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  max_vals, max_indices = torch.max(out, 1)
  answer.append(max_indices.cpu().clone().numpy())
  test_acc += calc_accuracy(out, label)
print(test_acc / (batch_id+1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()



0.7601143292682927


In [ ]:
# 제출 형식에 맞춰 파일을 저장해줍니다
ls = []
for i in answer:
  ls.extend(i)

pred = pd.DataFrame(ls, columns=['Predicted'])#.to_csv('drive/My Drive/predict_mango.csv')
df = pd.concat([eval_data['Sentence'], pred['Predicted'], eval_data['emotion']], axis=1) 

def test(x):
  if x==0.0: return 'sad'
  elif x==1.0: return 'happy'
  elif x==2.0: return 'annoy'
  elif x==3.0: return 'fear'
  else: return x

df["Predicted"] = df["Predicted"].apply(test)
df["emotion"] = df["emotion"].apply(test)

In [ ]:
for i in ['sad','happy','annoy','fear']: 
  print(i)
  print('real',len(df[df['emotion'] == i]))
  print('predict',len(df[df['Predicted'] == i]))

sad
real 0
predict 1808
happy
real 2844
predict 2679
annoy
real 1150
predict 621
fear
real 1240
predict 126


In [ ]:
# sentence.to_csv('drive/My Drive/emotion_tweet.csv')

#4.2 새로운 시도 (2번 평가)
이번엔 키워드 3개만 사용하고 각 감정별 키워드 1개를 validation으로 사용

In [ ]:
sad_3 = sad[:1000]
happy_3 = happy[:1000]
annoy_3 = annoy[:2000]
sentence = pd.concat([sad_3, happy_3, annoy_3, fear], axis=0, ignore_index=True)

sentence_train = pd.concat([sentence, sad2[:-1000], annoy2[:-1000], happy2[:-1000], fear2[:-1000]], axis=0, ignore_index=True)
sentence_train =sentence_train[['raw_text','emotion']]
len(sentence_train)

13464

In [ ]:
# 각 키워드가 약 1000개로 이루어져 있기 때문에 마지막 1000개를 평가 데이터로 선택
sentence_eval = pd.concat([sad2[-1000:], annoy2[-1000:], happy2[-1000:], fear2[-1000:]], axis=0, ignore_index=True)
sentence_eval = sentence_eval[sentence_eval['emotion'] != '중립']
sentence_eval =sentence_eval[['raw_text','emotion']]
len(sentence_eval)

4000

In [ ]:
def label(x):
  if x=='슬픔': return 0.0
  elif x=='기쁨': return 1.0
  elif x=='분노': return 2.0
  elif x=='공포': return 3.0
  else: return x

sentence_train["emotion"] = sentence_train["emotion"].apply(label)

dtls = [list(sentence_train.iloc[i,:]) for i in range(len(sentence_train))]
dtls[:10] #형식이 통일되었습니다



In [ ]:
#학습 과정
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad() 
        token_ids = token_ids.long().to(device) 
        segment_ids = segment_ids.long().to(device) 
        valid_length= valid_length 
        label = label.long().to(device) 
        out = model(token_ids, valid_length, segment_ids) 
        loss = loss_fn(out, label) 
        loss.backward() 
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) 
        optimizer.step() 
        scheduler.step()  # Update learning rate schedule 
        train_acc += calc_accuracy(out, label)  
        if batch_id % log_interval == 0: 
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1))) 
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1))) 
    model.eval() #모델 평가 부분 
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


epoch 1 batch id 1 loss 1.4466217756271362 train acc 0.171875

epoch 1 train acc 0.7465587797619048



epoch 1 test acc 0.9667002217553688


epoch 2 batch id 1 loss 0.20891311764717102 train acc 0.953125

epoch 2 train acc 0.9732142857142857



epoch 2 test acc 0.9752567693744164


epoch 3 batch id 1 loss 0.03695569932460785 train acc 0.984375

epoch 3 train acc 0.9867931547619048



epoch 3 test acc 0.9740458683473389


epoch 4 batch id 1 loss 0.008414948359131813 train acc 1.0

epoch 4 train acc 0.9936755952380952



epoch 4 test acc 0.9751619397759103


In [ ]:
##################################################
# 평가용 Test_set을 모델에 입력하기 위해 형식을 맞춰줍니다

def label(x):
  if x=='sad': return 0.0
  elif x=='happy': return 1.0
  elif x=='annoy': return 2.0
  elif x=='fear': return 3.0
  else: return x

sentence_eval["emotion"] = sentence_eval["emotion"].apply(label)
for i in range(len(sentence_eval['raw_text'])):
  if type(sentence_eval['raw_text'][i]) != str:
    sentence_eval['raw_text'][i] = str(sentence_eval['raw_text'][i])
dtls = [list(sentence_eval.iloc[i,:]) for i in range(len(sentence_eval))]
dtls[:10] #형식이 통일되었습니다

data_test = BERTDataset(dtls, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
#정확도를 계산하기 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
# 해당 데이터에 대해 분류를 시작합니다
model.eval()
answer=[]
train_acc = 0.0
test_acc = 0.0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  max_vals, max_indices = torch.max(out, 1)
  answer.append(max_indices.cpu().clone().numpy())
  test_acc += calc_accuracy(out, label)
print(test_acc / (batch_id+1))

In [ ]:
# 제출 형식에 맞춰 파일을 저장해줍니다
ls = []
for i in answer:
  ls.extend(i)

pred = pd.DataFrame(ls, columns=['Predicted'])#.to_csv('drive/My Drive/predict_mango.csv')
df = pd.concat([eval_data['Sentence'], pred['Predicted'], eval_data['emotion']], axis=1) 

def test(x):
  if x==0.0: return 'sad'
  elif x==1.0: return 'happy'
  elif x==2.0: return 'annoy'
  elif x==3.0: return 'fear'
  else: return x

df["Predicted"] = df["Predicted"].apply(test)
df["emotion"] = df["emotion"].apply(test)

for i in ['sad','happy','annoy','fear']: 
  print(i)
  print('real',len(df[df['emotion'] == i]))
  print('predict',len(df[df['Predicted'] == i]))